# Alternate Macrophage Polarization Neural Net Evaluation

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
import torch
import seaborn as sn
from scipy import ndimage
import pickle
from os import walk
from sklearn.preprocessing import StandardScaler
from operator import mul
from functools import reduce
import copy
from sklearn.manifold import TSNE
from timm.models import create_model
from timm.data import ImageDataset, create_loader

# Confusion matrices

In [ ]:
data = pickle.load(open("..\\..\\data\\external\\mac_polar_test_res\\model_test_results.p", "rb"))
confusion_matrices = []
matrices = []
test_accuracies = []
num_folds = len(data)
arch_matrices = {}
for fold in range(num_folds):
    curr_data = list(zip(*data[fold]))
    data[fold] = {}
    data[fold]["true"] = curr_data[0]
    data[fold]["pred"] = curr_data[1]

    labels = data[fold]["true"]
    labels = torch.flatten(torch.stack(labels))

    outputs = data[fold]["pred"]
    num_classes = len(outputs[0][0])
    outputs = torch.flatten(torch.stack(outputs), end_dim=1)
    outputs = torch.argmax(outputs, 1)
    outputs = np.array(outputs)

    confusion_matrices.append(np.zeros((num_classes, num_classes))) 

    for t, p in zip(labels, outputs):
            confusion_matrices[fold][int(t), int(p)] += 1
    matrix = np.array(confusion_matrices[fold])
    matrix = np.array([i/sum(i) for i in matrix])
    matrices.append(matrix)

    test_accuracy = 0
    for i in range(num_classes):
        test_accuracy += confusion_matrices[fold][i,i]
    test_accuracy = test_accuracy / sum(confusion_matrices[fold].flatten())
    test_accuracies.append(test_accuracy)
arch_matrices["macnet"] = sum(confusion_matrices)

print(f"Average: Test Accuracy: {sum(test_accuracies)/num_folds:>.2%}")



In [ ]:
archs = [
    'efficientnet_b0', 
    'efficientnet_b2',
    'inception_v4', 
    'pnasnet', 
    'resnext'
]

base_path = '..\\..\\data\\external\\mac_polar_test_res\\'

data = {}
for arch in archs:
    arch_matrices 
    confusion_matrices = []
    matrices = []
    test_accuracies = []
    num_folds = 5
    data[arch] = []
    for fold in range(num_folds):
        path = "\\".join([base_path, arch, f"fold_{fold+1}"])
        data[arch].append(pd.read_csv(path,header=None, names=["label", "pred", "M0", "M1", "M2"]))
        labels = np.array(data[arch][fold]["label"])

        outputs = np.array(data[arch][fold]["pred"])
        num_classes = int(np.max(labels) + 1)

        confusion_matrices.append(np.zeros((num_classes, num_classes))) 

        for t, p in zip(labels, outputs):
                confusion_matrices[fold][int(t), int(p)] += 1
        
        matrix = np.array(confusion_matrices[fold])
        matrix = np.array([i/sum(i) for i in matrix])
        matrices.append(matrix)

        test_accuracy = 0
        for i in range(num_classes):
            test_accuracy += confusion_matrices[fold][i,i]
        test_accuracy = test_accuracy / sum(confusion_matrices[fold].flatten())
        test_accuracies.append(test_accuracy)
    data[arch] = pd.concat(data[arch])
    arch_matrices[arch] = sum(confusion_matrices)

    print(f"{arch} Test Accuracy (avg): {sum(test_accuracies)/num_folds:>.2%}")
archs.append('macnet')

In [ ]:
axis_labels = ["M0", "M1", "M2"]
num_classes = len(axis_labels)

for arch in archs:
    curr_matrix = arch_matrices[arch]
    curr_matrix_pct = np.array([i/sum(i) for i in curr_matrix])
    matrix_df = pd.DataFrame(curr_matrix_pct, index=axis_labels, columns=axis_labels)
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(matrix_df, annot=True, fmt='.2%') # font size
    plt.title(f'M0/M1/M2 Classification Using {arch}')
    plt.show()

# F1 Score


In [ ]:
roc_data = []
for arch in archs:
    matrix_count = arch_matrices[arch]
    FP = matrix_count.sum(axis=0) - np.diag(matrix_count)  
    FN = matrix_count.sum(axis=1) - np.diag(matrix_count)
    TP = np.diag(matrix_count)
    TN = matrix_count.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    # F1 score
    F1 =  2 * (PPV * TPR) / (PPV + TPR)
    #print(PPV, TPR, F1, ACC)


    columns = {"Sensitivity": np.mean(TPR), 
            "Precision": np.mean(TNR), 
            "F1": np.mean(F1), 
            "Accuracy": np.mean(ACC)}
    
    for key in columns.keys():
        type_data = key
        value = columns[key]
        roc_data.append([arch, type_data, value])
        
roc_data = pd.DataFrame(roc_data, columns=["class", "data_type", "value"])

In [ ]:
sn.set_theme(style="whitegrid")

g = sn.catplot(
    data=pd.DataFrame(roc_data), kind="bar",
    x="class", y="value", hue="data_type",
    ci="sd", palette="dark", height=6
)
g.set(ylim=(0, 1))
g.set_axis_labels("label", "Percentage")
g.legend.set_title("Metric")

# T-SNE 

Redefining standardization operation used in training because cannot get transforms.Compose to work properly with Dataset object. TODO: Fixable 

In [ ]:
def standardize_input(image):
    image = image.detach().numpy()
    chans = range(image.shape[1])
    means = [np.mean(image[0][chan]) for chan in chans]
    stdevs = [np.std(image[0][chan]) for chan in chans]
    for chan in chans:
        image[0][chan] = (image[0][chan] - means[chan]) / stdevs[chan]
    
    output = torch.Tensor(image)
    return output

## Calculate fluorescent intensities of stains

In [ ]:
def calculate_intensity(img, channel_num, diag=False):
    arr = np.copy(img[channel_num])
    avg1 = list(arr[:2,:].flatten())
    avg2 = list(arr[:,:2].flatten())
    avg3 = list(arr[-2:,:].flatten())
    avg4 = list(arr[:,-2:].flatten())
    avgs = avg1 + avg2 + avg3 + avg4
    avgs.sort()
    avg = avgs[int(len(avgs)*0.9)] # 70th percentile

    
    arr2 = np.copy(arr)
    arr2 = arr2 - avg
    arr2[arr2 < 0] = 0
    arr2 = ndimage.median_filter(arr2, size=3)
    num_non_zero = np.count_nonzero(arr2)
    num_total = np.sum(arr2)
    avg2 = num_total / num_non_zero
    
    lit_pct = num_non_zero/(96*96)*100

    if diag:
        print("average intensity of lit pixels: ", round(avg2,2))
        print("percentage \"lit\": ", round(lit_pct, 2))
        toshow = [arr, arr2]
        labels = ["Stain", "Clean Stain"]
        num_show = len(toshow)
        f, axarr = plt.subplots(1,num_show, figsize=(8, 4))
        for i in range(num_show):
            axarr[i].imshow(toshow[i])
            axarr[i].grid(False)
            axarr[i].set_title(labels[i]) 
            axarr[i].get_xaxis().set_visible(False)
            axarr[i].get_yaxis().set_visible(False)

        plt.show()
    return avg2

Get CD80 and CD206 images and calculate fluorescent intensity

Not in use

In [ ]:
if False:
    fluor_marks_temp = {}
    fluor_paths = {
        ('M0', r'../../data/processed/kerryn_dec/M0.pickle'),
        ('M1', r'../../data/processed/kerryn_dec/M1.pickle'),
        ('M2', r'../../data/processed/kerryn_dec/M2.pickle')
    }
    for ele in fluor_paths:
        data = pickle.load(open(ele[1], 'rb'))
        fluor_marks_temp[ele[0]] = data['images'][:,2:4,:,:]

    # Get sample order in each fold. This could be made cleaner by re-organizing everything from the start
    fluor_marks = {}
    for test_fold in range(1,6):
        test_path = '../../data/processed/dataset_split/fold_%i/test' % test_fold
        fluor_marks[test_fold] = {}
        for pheno in ['M0', 'M1', 'M2']: # Clean this up later
            fluor_marks[test_fold][pheno] = {}
            curr_path = f'{test_path}/{pheno}'
            (_, _, curr_files) = next(walk(curr_path))
            curr_files = [int(file.rstrip('.png')) for file in curr_files]
            for file_num in curr_files:
                fluor_marks[test_fold][pheno][file_num] = fluor_marks_temp[pheno][file_num]

    fluorescent_stain = {}
    thresh = 8
    # 0 for CD80(blue), 1 for CD206(red), 2 for both (purple), 3 for neither (grey)

    # Need to get these results per fold then apply it to the fold tested
    for test_fold in range(1,6):
        fluorescent_stain[test_fold] = {}

        for pheno in ['M0', 'M1', 'M2']: # Clean this up later
            fluorescent_stain[test_fold][pheno] = []

            for sample_num in fluor_marks[test_fold][pheno]:
                CD80_brightness = calculate_intensity(fluor_marks[test_fold][pheno][sample_num], 0)
                CD206_brightness = calculate_intensity(fluor_marks[test_fold][pheno][sample_num], 1)
                

                cond_1 = CD80_brightness > thresh
                cond_2 = CD206_brightness > thresh
                if cond_1 and cond_2:
                    fluorescent_stain[test_fold][pheno].append(2)
                elif cond_1:
                    fluorescent_stain[test_fold][pheno].append(0)
                elif cond_2:
                    fluorescent_stain[test_fold][pheno].append(1)
                else:
                    fluorescent_stain[test_fold][pheno].append(3)

# Test and Extract Feature Map
Define calculation for extracting CD80 and CD206 levels from stains. Corrective techniques applied to remove image noise and variance between light levels

In [ ]:
model_path = {}
model_path['efficientnet_b0'] = r'.\output\train\20220217-212448-efficientnet_b0-96\model_best.pth.tar' 
model_path['efficientnet_b2'] = r'.\output\train\20220217-222826-efficientnet_b2-96\model_best.pth.tar'
model_path['inception_v4'] =    r'.\output\train\20220218-000119-inception_v4-96\model_best.pth.tar'
model_path['pnasnet5large'] =   r'.\output\train\20220218-021523-pnasnet5large-96\model_best.pth.tar'
model_path['resnext101_32x8d'] =r'.\output\train\20220218-103054-resnext101_32x8d-96\model_best.pth.tar'

tsne_results = {}
base_path = '..\\..\\data\\external\\mac_polar_test_res\\'

for arch in archs:    
    print('Processing ' + arch)
    test_path = "\\".join([base_path, arch, f"fold_1"])
    num_classes = 3
    model = create_model(
        arch,
        num_classes=num_classes,
        in_chans=3,
        pretrained=False,
        checkpoint_path=model_path[arch])
    model = model.cuda()

    loader = create_loader(
        ImageDataset(test_path),
        input_size=(3,96,96),
        batch_size=1,
        use_prefetcher=True,
        interpolation='bicubic',
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
        num_workers=1,
        no_aug=True
        )
    model.eval()
        
    feature_maps = []
    targets = []
    with torch.no_grad():
        for batch_idx, (input, target) in enumerate(loader):
            input = input.cuda()
            feature_maps.append(model.forward_features(input).cpu().numpy())
            targets.append(target.cpu())

    feature_maps = np.stack(feature_maps)
    num_samples = feature_maps.shape[0] * feature_maps.shape[1]
    features = feature_maps.shape[2:]
    feature_maps = np.reshape(feature_maps, (num_samples,) + features)
    feature_maps = np.reshape(feature_maps, (num_samples, reduce(mul, features)))
    
    targets = np.stack(targets)
    targets = np.reshape(targets, (num_samples))

    scaled_data = StandardScaler().fit_transform(feature_maps)
    TSNE = TSNE(n_components=2, perplexity=40, n_iter=10000, learning_rate=200)
    tsne_results = TSNE.fit_transform(scaled_data)
    df_tsne = pd.DataFrame(tsne_results, columns=['t-sne-one', 't-sne-two'])
    phenotypes = {0:"M0", 1:"M1", 2:"M2"}
    df_tsne['label'] = [phenotypes[int(ele)] for ele in targets]
    tsne_results[arch] = df_tsne


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

num_archs = len(archs)
# fig, ax = plt.subplots(1, num_archs, figsize=(10*num_archs+5,10))


for i, arch in enumerate(archs):
    fig, ax =plt.subplots(1,2, figsize=(20,10))

    colors = ["#0a70c4", "#db0d0d", "#660ddb", "#b5b5b5"]
    customPalette = sns.set_palette(sns.color_palette(colors))
    sns.scatterplot(
        x="umap-one", y="umap-two",
        hue="fluor_mark",
        palette=customPalette,
        data=umap_results[arch],
        hue_order = ['CD80+', 'CD206+', 'CD80+/CD206+', 'CD80-/CD206-'],
        legend="full",
        alpha=0.5,
        ax=ax[0],
    )
    ax[0].set_title("Fluorescent Marker Phenotype")
    colors = ["#b5b5b5", "#0a70c4", "#db0d0d"]
    customPalette = sns.set_palette(sns.color_palette(colors))         
    sns.scatterplot(
        x="umap-one", y="umap-two",
        hue="label",
        data=umap_results[arch],
        palette=customPalette,
        hue_order = ['M0', 'M1', 'M2'],
        legend="full",
        alpha=0.5,
        ax=ax[1]
    )
    ax[1].set_title("UMAP for " + arch)
    plt.show()